# Definir archivos de lectura de datos y output 

In [2]:
data_train="input/datacontinuousstudents.csv"
data_test="testContinua.csv"
nombre_output='output_continua_GP.csv'

# Librerias necesarias

In [3]:
import pandas as pd
import numpy as np
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel


# Lectura datos entrenamiento

In [4]:
data=pd.read_csv(data_train)

# Entrenamiento Modelo

In [10]:
sels=['x13','x23','x31','x10','x25']

def stan(df,dummies):
    d2 = {df.columns[0]:list(df[df.columns[0]])}
    for var in df.columns[1:]:
        #print(var)
        if var in dummies:
            aux = df[var]
        else:
            median = np.median(df[var])
            desv = np.std(df[var])
            aux = df[var].apply(lambda x: (x-median)/desv)
        d2[var] = list(aux)
    data2 = pd.DataFrame.from_dict(d2)
    
    return data2

dummies = ['x3','x4','x5','x6','x7','x13','x14','x15','x16','x17','x18','x19','x20','x28','x29']

data2 = stan(data,dummies)

X_train=data2.filter(sels, axis=1)
Y_train=data2['y']


kernel = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=1)

final_model = gaussian_process.GaussianProcessRegressor(kernel=kernel)
final_model.fit(X_train, Y_train)
    
sels.append('int')


# Testeo

In [13]:
test=pd.read_csv('testContinua.csv')
sels=['x13','x23','x31','x10','x25']
y_pred_test, sigma = final_model.predict(test.filter(sels, axis=1), return_std=True)
pd.DataFrame({'y_hat':y_pred_test}).to_csv(nombre_output)